In [10]:
import gurobipy as gp
import time
from gurobipy import GRB, quicksum
import numpy as np

In [17]:
d=[15,23,26,12,31,19,25,17,21,14]
c=[[1.3,0.7,0.1,0.3,0,0.9,1,0.1,0.3,0.1],
   [0, 0.2, 0.3, 0.2, 0.9, 0.4, 0.2, 0.8, 0.1, 0.6],
   [0.4, 0.3, 1.5, 0, 0.5, 0.5, 0.2, 0.4, 1.0, 0.4],
   [0.5, 0.5, 0.2, 0.1, 0.7, 0.8, 0.8, 0.2, 0, 1.1],
   [0.2, 0.3, 0.1, 0.7, 0.2, 0.9, 0.4, 0.6, 0.3, 1.4]
 ]
f=20
C=100
I=10
J=5
y=[1,1,1,1,1]
P=[]
R=[]
UB=10000
LB=-10000

In [18]:
tic=time.perf_counter()
inti=0
while LB-UB>=0.001 or UB-LB>=0.0001:
    
    be=gp.Model()
    x=be.addVars(I,J ,vtype=GRB.CONTINUOUS ,name='x')

    objective=0
    for i in range(I):
        for j in range(J): 
            objective+=c[j][i]*d[i]*x[i,j]
    be.setObjective(objective, GRB.MINIMIZE)
    for i in range(I):     
        be.addConstr(quicksum(x[i,j] for j in range(J))==1)
    for j in range(J):
        be.addConstr(quicksum(d[i]*x[i,j] for i in range(I))<=C*y[j])

    be.Params.InfUnbdInfo=1
    be.Params.dualReductions=0
    be.optimize()
    try:
        tr=be.UnbdRay
        print('unbounded')

    except:
        if be.status ==2 :
            print('Feasible. Optimality cut obtained')
            a=be.pi
            P.append(a)
        elif be.status== 3:
            print('Unbounded. Feasibility cut obtained')
            a=be.pi
            R.append(a)
    print(be.pi)
    sumy=0
    for i in y:
        sumy+=i*f
    print(UB)
    if UB>be.objVal:
        UB=be.objVal+sumy
    print(UB)

    BMP1=gp.Model()
    theta=BMP1.addVar(vtype=GRB.CONTINUOUS, name='theta')

    y=BMP1.addVars(J,vtype=GRB.BINARY, name='y')
    BMP1.setObjective(quicksum(y[j]*f for j in range(J))+theta, GRB.MINIMIZE)
    #for sub1


    for p in P:
        const=0
        for i in range(I):
            const+=p[i]
        for j in range(I,I+J):
            const+=C* y[j-I]*p[j]
        BMP1.addConstr(const<=theta)
    for r in R:
        const=0
        for i in range(I):
            const+=r[i]
        for j in range(I,I+J):
            const+=C* y[j-I]*r[j]
        BMP1.addConstr(const<=0)


    BMP1.optimize()
    BMP1.printAttr('X')
    if LB<BMP1.objVal:
        LB=BMP1.objVal



    ys=[]
    for i in range(J):
        ys.append(y[i].x)
    y=ys
    print(y)
    inti+=1

toc=time.perf_counter()
print('Objective Val' , LB)
print('Runtime ' ,toc-tic)
print('No of iterations', inti)

Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter dualReductions to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 15 rows, 50 columns and 100 nonzeros
Model fingerprint: 0x89960307
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve time: 0.00s
Presolved: 15 rows, 50 columns, 100 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2900000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  2.290000000e+01
Feasible. Optimality cut obtained
[-0.0, 4.6000000000000005, 2.6, -0.0, -0.0, 7.6000000000000005, 5.0, 1.7000000000000002, -0.0, 1.4000000000000001, 0.0, 0.0, 0.0, 0.0, 0.0]
1000

        y[3]            1 
[1.0, 1.0, 0.0, 1.0, 0.0]
Objective Val 84.1
Runtime  0.11327939998591319
No of iterations 4


In [4]:
xs=[]
for i in range(I):
    for j in range (J):
        xs.append(x[i,j].x)
xs=np.array(xs)
xs=xs.reshape(I,J).T
xs


array([[0., 0., 1., 0., 1., 0., 0., 1., 0., 1.],
       [1., 1., 0., 0., 0., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])